<a href="https://colab.research.google.com/github/TSA-bit/big-data-challenge/blob/main/Level1_luggage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [3]:
#connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-07 02:18:47--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.71MB/s    in 0.2s    

2022-03-07 02:18:48 (5.71 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
#start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon_reviews_us_Pet_Products").config("apark.driver.extraClassPath", "/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
#Read in data from S3 buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), inferSchema=True, sep="\t")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [7]:
#check datatypes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
#change review_date to date format
from pyspark.sql.types import DateType

df = df.withColumn("review_date", df.review_date.cast(DateType()))
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [13]:
#drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

348657
348613
348613


In [14]:
#verify data types after changes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [15]:
#create review_id_table
reviews_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11IBSD5E6HPSD|   20761040|B002B3FWXY|     677901073| 2015-08-31|
|R16F6MZAVR1IYL|   29442013|B000YB7T04|     553070376| 2015-08-31|
|R30889FXLP48F3|   29849475|B00F9KE912|     918753137| 2015-08-30|
|R1DW06GLTNCVWQ|    1988134|B001CDG7Q0|     460451174| 2015-08-29|
| RDC197RTSTZJY|    1302634|B0019GAOO6|      53787504| 2015-08-29|
|R3D44CQ92OA3BR|   26953574|B00YGDX2XE|     613329783| 2015-08-29|
|R115NO9WEEUBWD|   38639848|B00KH5FB5W|     684605802| 2015-08-28|
|R16QUOK50Y94JU|    8004704|B00JKJQJHA|     860629696| 2015-08-28|
| R1P81BC6VL7PZ|     121815|B00WJJVXQU|     990303407| 2015-08-28|
| RZZLHN9MWIAEJ|   48578265|B006GN4ERU|     333298218| 2015-08-28|
|R1A75HAI9Q04WV|    1211291|B00Z9NV81I|      63474760| 2015-08-27|
|R1PCHIM41UIET7|   32335114|B001TQTWGW|     293242998| 2015-08

In [16]:
#create product table
product_df = df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002B3FWXY|Travelon Anti-The...|
|B000YB7T04|totes Signature C...|
|B00F9KE912|Samsonite Wheeled...|
|B001CDG7Q0|17 Inch Gray Whee...|
|B0019GAOO6| Everest Hiking Pack|
|B00YGDX2XE|MEKU PU leather P...|
|B00KH5FB5W|Ever Moda Designe...|
|B00JKJQJHA|Dot&Dot Travel Ac...|
|B00WJJVXQU|17" Red & Black M...|
|B006GN4ERU|Olympia Deluxe Fa...|
|B00Z9NV81I|SENPAIC Canvas La...|
|B001TQTWGW|Travelon Anti-The...|
|B009AZ69V0|Traveler's Choice...|
|B00FF4AAS8|HOVEOX Stylish Me...|
|B000BT2FK8|Royce Leather Lux...|
|B00DRJQZ0K|Reinforced Design...|
|B011VZE7DS|Monster Trucks Ba...|
|B00LZXPJB6|Canvas 15" Laptop...|
|B00FUWSU3W|Bago Travel Duffe...|
|B00LJUROJ0|Travelpro Infligh...|
+----------+--------------------+
only showing top 20 rows



In [18]:
#count total number rows in product_df
counted = product_df.count()
print(f"Total number product: {counted}")

Total number product: 348613


In [19]:
#product table needs only unique values
product_df = product_df.distinct()

#check new row count
recounted = product_df.count()
print(f"Total number after recount: {recounted}")

Total number after recount: 64584


In [21]:
#create customers table, per schema need customer count
customers_df = df.groupBy("customer_id").count()

#rename count column
customers_df = customers_df.withColumnRenamed("count", "customer_count")

#order by customer count
customers_df.orderBy(customers_df["customer_count"].desc()).show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12640280|            63|
|   16600581|            31|
|   27493011|            31|
|   43856165|            29|
|   39789300|            26|
|   35461976|            25|
|   35938574|            24|
|   52955780|            23|
|    5329443|            23|
|   52228204|            23|
|   37244426|            22|
|   49266466|            21|
|   51712970|            19|
|   47169566|            19|
|   20844007|            19|
|     497832|            19|
|   51346302|            18|
|   11422115|            17|
|   10009912|            17|
|   14866545|            17|
+-----------+--------------+
only showing top 20 rows



In [22]:
#check data type for customers_df
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [23]:
#per schema customer count needs to be integer
from pyspark.sql.types import IntegerType

customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))

#verify data type change
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [24]:
#Create vine_table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R11IBSD5E6HPSD|          1|           29|         31|   N|
|R16F6MZAVR1IYL|          4|            0|          0|   N|
|R30889FXLP48F3|          5|            1|          1|   N|
|R1DW06GLTNCVWQ|          5|            0|          0|   N|
| RDC197RTSTZJY|          4|            0|          0|   N|
|R3D44CQ92OA3BR|          5|            0|          0|   N|
|R115NO9WEEUBWD|          4|            1|          1|   N|
|R16QUOK50Y94JU|          5|            0|          0|   N|
| R1P81BC6VL7PZ|          4|            0|          0|   N|
| RZZLHN9MWIAEJ|          5|            0|          0|   N|
|R1A75HAI9Q04WV|          1|            8|          9|   N|
|R1PCHIM41UIET7|          3|            0|          0|   N|
|R2S1PWOTIMYQ29|          5|            0|          1|   N|
|R3MFI5JTNYZCFG|          5|            

In [25]:
#configure settings for RDS
mode = "append"
jdbc_url = "jdbc:postgresql://<connection string>:5432/<luggage>"
config = {"user":"<USERNAME>", 
          "password": "<PASSWORD>", 
          "driver":"org.postgresql.Driver"}


In [ ]:
#write dataframe to review_id table in RDS
reviews_df.write.jdbc(url=jdbc_url, table="review_id", mode=mode, properties=config)

In [ ]:
#write dataframe to product table in RDS
product_df.write.jdbc(url=jdbc_url, table="product", mode=mode, properties=config)

In [ ]:
#write dataframe to customers in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
#write dataframe to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)